In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.modeling import models
import pandas as pd
import os
import spfuncs as spf
from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy.nddata import StdDevUncertainty,InverseVariance
from scipy.io import readsav
from scipy.stats import chi2
from scipy.interpolate import UnivariateSpline,CubicSpline
from scipy.optimize import curve_fit
from IPython.display import clear_output
from astropy.convolution import Gaussian1DKernel, convolve_fft
from scipy.stats import binned_statistic
from specutils import Spectrum1D,SpectralRegion
from specutils.fitting import fit_lines
from specutils.manipulation import FluxConservingResampler
from specutils.manipulation.model_replace import model_replace,extract_region
from specutils.analysis import line_flux
from matplotlib.lines import Line2D
import dust_extinction

# GLOBAL VARIABLES

## Constants

In [3]:
########### CONSTANTS ##############
c = 3e18 # A/s
c_kms = 3e5 # km/s
h = 4.1357e-18 # keV s
erg = 1/6.2415e8 # 1 erg = 6.2e8 keV
pc2au = 206265 # au/pc
####################################

# global variable for instrument bits
INST_BITS = {'G140L':1,
             'G140M':2,
             'G230L':4,
             'G430L':8,
             'X-RAY':16, # Chandra or XMM-Newton
             'proxy':32,
             'lya':64,
             'EUV':128,
             'BT-Settl':256,
             'APEC':512,
             'E230M':1024, #HD 149026
             'G130M':2048, #L 678-39
             'G160M':4096} #L 678-39

# FUNCTIONS

In [4]:
# Search for a spectrum in the given grating directory. If found, updates given grating dict and returns true
# Requires spectrum be named x1d_coadd.fits
def look_for_data(star,grating_to_find,grating_dict):
    # Search through the directories in the Star directory and try to find the given grating
    
    print('Searching for',grating_to_find)
    try:
        for root,dirs,files in os.walk('./%s'%(star)):
            if(root == './%s\\%s'%(star,grating_to_find)):
                print('Found directory: %s'%(root))
                # Try to find the coadded spectrum in the grating directory
                try:
                    with fits.open('%s/x1d_coadd.fits'%(root)) as hdul:
                        wave = hdul[1].data['WAVELENGTH']
                        flux = hdul[1].data['FLUX']
                        err = hdul[1].data['ERROR']
                        inst = INST_BITS[grating_to_find]*np.ones(len(wave))
                    print('Retrieved',grating_to_find)
                    grating_dict[grating_to_find] = np.array([wave,flux,err,inst])
                    return True
                except:
                    print('Error finding spectrum')
    except:
        print('Error finding star directory')
    
    print('Did not find directory')
    return False

# Make an emission line given the line center and integrated line flux
def make_emission_line(line_center,flux_to_match):
    # INPUT:
    # line_center: center of emission line in angstroms]
    # flux_to_match: integrated line flux
    # OUTPUT:
    # spectrum: A 1d spectrum of the created emission line to be insert into the final combined spectrum
    # width: width of the emission line in Å
    # best_amp: The amplitude of the best-fit gaussian model
    # int_flux: the integrated flux of the best-fit gaussian model
    
    
    # Creates a new wavelength grid assuming that +/-3A is large enough to cover the entire line (should be /shrug)
    new_wave_grid = np.linspace(line_center-1.5,line_center+1.5,num=1000)
    
    # Converts to velocity space to calculate the FWHM
    v_array = (new_wave_grid-line_center)/new_wave_grid*c_kms
        
    # Set width to 60 or 70 km/s based on spectral type.
    # Estimations from France(2020) Barnard * paper & France (2010) HD209458 paper
    if(STELLAR_TYPE.lower()=='m'):
        width = 60 # km/s    
    else:
        width = 70 # km/s
    
    lower_bound,upper_bound = np.where(np.abs(v_array) <= width)[0][0],np.where(np.abs(v_array) <= width)[0][-1]
    
    # Convert the FWHM back to wavelength space
    FWHM_wave = new_wave_grid[upper_bound]-new_wave_grid[lower_bound]
    
    # Create an array of amplitudes to iterate over
    amp_array = np.linspace(flux_to_match*1e-1,flux_to_match*1e1,num=10000)
    
    best_amp = 0
    int_flux = 0
    
    # Find the amplitude that makes the best-fit gaussian model by matching the integrated flux_to_fit
    for amp in amp_array:
        #print('Amplitude:',amp) # debugging
        gaussian = models.Gaussian1D(amplitude=amp,mean=line_center,stddev=FWHM_wave/2.355)
        model = gaussian(new_wave_grid)
        int_flux = np.trapz(x=new_wave_grid,y=model)
        #print('Flux:',int_flux) # debugging
        best_amp = amp
        if(int_flux >= flux_to_match):
            break
    
    return new_wave_grid,model,best_amp,int_flux

def minimize_scale_factor(ref,spectrum,init_factor,search_range):
    # Minimizes the scale factor [(F_obs - F_proxy)/E_obs]^2. Returns the best scale factor 
    #      and the minimum "chi^2" value
    # INPUT:
    # ref = reference spectrum. numpy array including wavelength, flux, error [wave,flux,err]
    # spectrum = spectrum to be scaled. 1d numpy array containing ONLY flux.
    # init_factor = initial scaling factor
    # search_range = order of magnitude to search around init_factor. ie: search_range = 2 will search for 
    #                factors between init_factor*10^(-2)-init_factor*10^(+2)
    # OUTPUT:
    # alpha scaling value and minimum "chi squared" factor
    
    scale_factors = np.linspace(init_factor*10**(-search_range),init_factor*10**(search_range),num=5000)
    #print('Scale factor lims',min(scale_factors),max(scale_factors)) # debugging
    best_factor = 1
    min_chi2 = np.inf
    
    for alpha in scale_factors:
        scaled_flux = spectrum*alpha
        chi2 = np.sum(((ref[1]-scaled_flux)/ref[2])**2)/(len(ref[0])-1)
        
        if(chi2 < min_chi2):
            min_chi2 = chi2
            best_factor = alpha
            
    return best_factor,min_chi2

def scale_spectrum(ref,spectrum,init_factor=1,coarse_range=2,fine_range=1,end='high'):
    # Takes a reference spectrum and a spectrum to be scaled. Calls minimize_scale_factor and returns the best 
    #        scale factor to match the spectrum to the reference.
    # INPUT:
    # ref, spectrum, init_factor = same as minimize_scale_factor()
    # coarse_range = value of search_range to be passed to minimize_scale_factor() for the initial search.
    #                This is intended to search a braod range to avoid local minima
    # fine_range = value of search_range to be passed to minimize_scale_factor() for the 2nd search.
    #               This is intended to find a more precise minimum around the result from the coarse search
    # end = which end of the REFERENCE spectrum are you scaling to. Must be 'low' or 'high' for blue 
    #               and red end respectively. 
    #               For BT-settl we scale the blue end of the spectrum to the red end of reference: end='high'
    #               For proxy we scale the red end of spectrum to the blue end of G230L: end='low'
    # OUTPUT:
    # scaled spectrum in form [wavelength, flux*alpha, error*alpha]
    # alpha scaling factor
    # minimum "chi squared" factor
    
    # Get the region where the spectrum and reference overlap
    if(end.lower()=='high'):
        try:
            ref_reg = ref[0] > spectrum[0][0]
            spec_reg = spectrum[0] < ref[0][-1]
            
            ref = ref[:,ref_reg]
            spectrum = spectrum[:,spec_reg]
        except:
            raise IndexError('For end=\'high\' the spectrum to be scaled must overlap with the red end of the reference')
    elif(end.lower()=='low'):
        try:
            ref_reg = ref[0] < spectrum[0][-1]
            spec_reg = spectrum[0] > ref[0][0]
            
            ref = ref[:,ref_reg]
            spectrum = spectrum[:,spec_reg]
        except:
            raise IndexError('For end=\'low\' the spectrum to be scaled must overlap with the blue end of the reference')
    else:
        raise Exception('Keyword \'end\' must be \'high\' or \'low\'')
        
    # print the overlap region used in the scaling routine
    print('Fitting region:\t%0.1f-%0.1f'%(min(spectrum[0]),max(spectrum[0])))
        
    # Interpolate the spectrum with greater A/pix onto the wavelength grid of the other
    ref_ang_per_pix = np.nanmean(np.diff(ref[0])/len(ref[0]))
    spectrum_ang_per_pix = np.nanmean(np.diff(spectrum[0])/len(ref[0]))
    
    if((ref_ang_per_pix < spectrum_ang_per_pix) or (ref_ang_per_pix == spectrum_ang_per_pix)):
        spectrum_interp = np.interp(x=ref[0],xp=spectrum[0],fp=spectrum[1])
        
        # Run the minimization routine with a broad, coarse search
        scale_factor_coarse,_ = minimize_scale_factor(ref,spectrum_interp,init_factor,search_range=coarse_range)
        # Run the minimization routine with a fine search around the initial factor
        scale_factor,chi2 = minimize_scale_factor(ref,spectrum_interp,scale_factor_coarse,search_range=fine_range)
        
    elif(spectrum_ang_per_pix < ref_ang_per_pix):
        ref_interp = np.interp(x=spectrum[0],xp=ref[0],fp=ref[1])
        ref_err_interp = np.interp(x=spectrum[0],xp=ref[0],fp=ref[2])
        
        ref_to_pass = np.array([spectrum[0],ref_interp,ref_err_interp])
        
        # Run the minimization routine with a broad, coarse search
        scale_factor_coarse,_ = minimize_scale_factor(ref_to_pass,spectrum[1],init_factor,search_range=coarse_range)
        # Run the minimization routine with a fine search around the initial factor
        scale_factor,chi2 = minimize_scale_factor(ref_to_pass,spectrum[1],scale_factor_coarse,search_range=fine_range)
        
    print('--- RESULTS ---')
    print('Best scale factor:\t%0.4e'%(scale_factor))
    print('Minimum chi square:\t%0.4e'%(chi2))
    
    return scale_factor,chi2

# Bin the instrument bits. After binning the spectrum, if there are data in a single bin coming from multiple sources
#     get_inst_bins() will add the instrument bits to indicate the separate sources
def get_inst_bins(inst_array):
    # INPUT: 1D instrument bit array from the spectrum being binned
    # OUTPUT: scalar value representing the sum of the instrument bits in the bin
    #             ie: A bin containing both proxy & G230L data will read 36 (4+32)
    bin_val = np.sum(np.unique(inst_array))
    return bin_val

# Star & Proxy Parameters

In [5]:
### Update ###
STAR = 'WASP-17'.upper()
PROXY_NAME = 'Procyon'
STELLAR_TYPE='F'
STELLAR_RADIUS_SOL = 1.573
STELLAR_DIST_PC = 410

### Do not update ###
SOLAR_DIST_PC = 4.8481e-6

# Read FITS files from star directory

In [6]:
grating_exists = {'G130M':False,'G140L':False,'G140M':False,'G160M':False,'G230L':False,'E230M':False,'G430L':False,}
gratings = {}

# Get STIS spectra
for grating_name in grating_exists.keys():
    # Search for spectra from the appropriate grating and add it to the grating dictionary
    grating_exists[grating_name] = look_for_data(STAR,grating_name,gratings)
    
# Retrieve proxy star spectrum
try:
    if(PROXY_NAME.lower()=='sun' or PROXY_NAME.lower()=='solar'):
        solar_dat = readsav('./Proxy stars/Solar/solar-data.idlsav')
        solar_wave = solar_dat['wave'][:,0]*10
        solar_flux = solar_dat['flux'][:,0]*100
        inst = INST_BITS['proxy']*np.ones(len(solar_wave))
        
        proxy = np.array([solar_wave,solar_flux,np.zeros(len(solar_wave)),inst])
        print('Retrieved Solar proxy')
    elif(PROXY_NAME.lower()=='procyon'):
        with fits.open('./Proxy stars/Procyon/Procyon_SED_1Ang_5um.fits') as hdul:
            wave = hdul[1].data['WAVE']
            flux = hdul[1].data['FLUX']
            err = np.zeros(len(wave))
            inst = np.ones(len(wave))*INST_BITS['proxy']
            
            proxy = np.array([wave,flux,err,inst])
            hdul.close()
            
        print('Rerieved proxy')
    else:
        for root,dirs,files in os.walk('./Proxy stars'):
            if(root == './Proxy stars\\%s'%(PROXY_NAME)):
                print('Found directory: %s'%(root))
                
                with fits.open('./Proxy stars/%s/%s'%(PROXY_NAME,files[0])) as hdul:
                    wave = hdul[1].data['WAVELENGTH']
                    flux = hdul[1].data['FLUX']
                    err = hdul[1].data['ERROR']
                    inst = INST_BITS['proxy']*np.ones(len(wave))
                    
                    proxy = np.array([wave,flux,err,inst])
                    hdul.close()
                    
                print('Retrieved proxy')
except:
    print('Could not retrieve proxy')
    
print('\nGratings list:')
print(grating_exists)

Searching for G130M
Did not find directory
Searching for G140L
Found directory: ./WASP-17\G140L
Retrieved G140L
Searching for G140M
Did not find directory
Searching for G160M
Did not find directory
Searching for G230L
Found directory: ./WASP-17\G230L
Retrieved G230L
Searching for E230M
Did not find directory
Searching for G430L
Found directory: ./WASP-17\G430L
Retrieved G430L
Rerieved proxy

Gratings list:
{'G130M': False, 'G140L': True, 'G140M': False, 'G160M': False, 'G230L': True, 'E230M': False, 'G430L': True}


#### Special cases: ie: COS, uncommmon STIS gratings, etc, do manually

## HD-149026 E230M spectrum ##
# We're just gonna go ahead and replace gratings['G230L'] with E230M data. 
# I realize this is a bad way to do it
# It's a one-off case, it'll be ok

hd_dat = pd.read_csv('./HD-149026/E230M/HD-149026_E230M_x1d.csv')
E230M_spectrum = np.array([hd_dat['wavelength'],hd_dat['flux'],hd_dat['error'],np.ones(len(hd_dat['flux']))*INST_BITS['E230M']])

E230M_conv_flux = spf.convolve_gaussian(E230M_spectrum[0],E230M_spectrum[1],1.58)
E230M_conv_err = spf.convolve_gaussian(E230M_spectrum[0],E230M_spectrum[2],1.58)
E230M_spectrum[0],E230M_spectrum[1] = E230M_conv_flux[0],E230M_conv_flux[1]
E230M_spectrum[2] = E230M_conv_err[1]

gratings['G230L'] = E230M_spectrum
print(gratings)

## L 678-39 COS data

In [ ]:
plt.figure()
plt.plot(gratings['G160M'][0],gratings['G160M'][1])

# Examine data and select good regions

In [7]:
for grating_name in gratings:
    print('Working with',grating_name)
    grating = gratings[grating_name]
    
    # Get good error region for plottign SNR
    good_err_reg = (grating[2] != 0)
    
    fig,ax = plt.subplots(2,1,figsize=(10,8),tight_layout=True,sharex=True,num=grating_name)
    fig.suptitle(grating_name)
    ax[0].plot(grating[0],grating[1])
    ax[1].plot(grating[0][good_err_reg],grating[1][good_err_reg]/grating[2][good_err_reg])
    ax[1].hlines(y=3,xmin=min(grating[0]),xmax=max(grating[0]),color='k',linestyle='--')
    ax[1].set_ylim([-3,20])
    
    ax[1].set_xlabel('Wavelength [A]')
    ax[0].set_ylabel('Flux [erg / s / cm^2 / A]')
    ax[1].set_ylabel('S/N')
    
    clicks = np.array(fig.ginput(n=-1,timeout=0))[-2::][:,0]
    
    selected_region = (grating[0] > clicks[0]) & (grating[0] < clicks[1])
    print('Keeping wavelengths from:  %0.2f-%0.2f'%(grating[0][selected_region][0],grating[0][selected_region][-1]))
    
    gratings[grating_name] = grating[:,selected_region]

Working with G140L
Keeping wavelengths from:  1683.50-1711.50
Working with G230L
Keeping wavelengths from:  1742.00-3163.50
Working with G430L
Keeping wavelengths from:  3172.00-5699.50


## Retrieve Ly$\alpha$ profile & BT-Settl

In [8]:
# Lya
try:
    lya_data = pd.read_csv('./%s/lya_reconstruction/%s_G140M_MCMC_results.csv'%(STAR,STAR))
    lya_err = np.mean((lya_data['lya_intrinsic_low_1sig'],lya_data['lya_intrinsic_high_1sig']),axis=0)
    lya = np.array([lya_data['wave_lya'],
                    lya_data['lya_intrinsic_median'],
                    lya_err,
                    np.ones(len(lya_data['wave_lya']))*INST_BITS['lya']])
    print('Retrieved MCMC reconstruction')
except:
    try:
        lya_data = pd.read_csv('./%s/lya_reconstruction/%s_lya_intrinsic_profile.csv'%(STAR,STAR))
        lya = np.array([lya_data['wave'],
                        lya_data['flux'],
                        np.zeros(len(lya_data['wave'])),
                        np.ones(len(lya_data['wave']))*INST_BITS['lya']])
        print('Retrieved Mg II reconstruction')
    except:
        print('No Lyman-alpha profile found')
        
# BT-Settl
try:
    settl_dat = Table.read('./%s/interp_spectra/%s_phoenix_interpolated.ecsv'%(STAR,STAR))
    settl = np.array([settl_dat['WAVELENGTH'],
                      settl_dat['FLUX']*settl_dat.meta['NORMFAC'],
                      np.zeros(len(settl_dat['FLUX'])),
                      np.ones(len(settl_dat['WAVELENGTH']))*INST_BITS['BT-Settl']])
    print('Retrieved BT-Settl')
except:
    print('No SETTL data found')

Retrieved Mg II reconstruction
Retrieved BT-Settl


# Convolve & Scale BT-Settl

#### Convolve BT-Settl to G430L Resolution

In [9]:
# Convolve BT-Settl to G430L resolution (~2.73A/pix [STIS handbook])
settl_conv_flux = spf.convolve_gaussian(settl[0],settl[1],2.73)
settl_conv = np.array([settl_conv_flux[0],settl_conv_flux[1],np.zeros(len(settl_conv_flux[0]))])

#### Scale convolved BT-Settl to G430L data

In [10]:
# Select the region of the settl spectrum to scale and run the minimization routine. Recommend >5000A
settl_to_fit = settl_conv[0] > 5000

settl_alpha,settl_chi2 = scale_spectrum(ref=gratings['G430L'],spectrum=settl_conv[:,settl_to_fit],
                                                init_factor=1,coarse_range=2,fine_range=0.5,end='high')

scaled_settl = np.array([settl_conv[0],
                         settl_conv[1]*settl_alpha,
                         settl_conv[2]*settl_alpha,
                         np.ones(len(settl_conv[0]))*INST_BITS['BT-Settl']])

fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True);
spf.setup_axis(ax)
ax.plot(gratings['G430L'][0],gratings['G430L'][1]*1e14,label='G430L');
ax.plot(settl_conv[0],settl_conv[1]*settl_alpha*1e14,label=r'Scaled BT-Settl: $\alpha$=%0.4f'%(settl_alpha),zorder=-1);
#ax.plot(settl_conv[0],settl_conv[1],label='Unscaled SETTL');
ax.legend(fontsize=12);

ax.set_xlim([min(gratings['G430L'][0])-300,max(gratings['G430L'][0])+500]);
ax.set_xlabel(r'Wavelength [$\AA$]',fontsize=14);
ax.set_ylabel(r'Flux density [10$^{-14}$ erg s$^{-1}$ cm${-2}$ Å$^{-1}$]',fontsize=14);

plt.savefig('./%s/bt-settl_scaling.png'%(STAR))

Fitting region:	5002.1-5697.9
--- RESULTS ---
Best scale factor:	6.9988e-01
Minimum chi square:	4.2409e+01


#### Scale proxy spectrum to G230L

In [11]:
# Select the region of the proxy spectrum to scale and run the minimization routine, assuming we have a G230L spectrum
# IGNORES MgII LINE
grating_exists['G230L'] = True
if(grating_exists['G230L']):
    proxy_to_fit = (proxy[0] > gratings['G230L'][0][0]) & (proxy[0] < 3000)
    mg2_reg = [2770,2820]
    
    proxy_mg = (proxy[0] > mg2_reg[0]) & (proxy[0] < mg2_reg[1])
    stis_mg = (gratings['G230L'][0] > mg2_reg[0]) & (gratings['G230L'][0] < mg2_reg[1])
    

    if((PROXY_NAME.upper()=='SUN') or (PROXY_NAME.upper()=='SOLAR')):
        
        # If the proxy is the Sun, scale it by (d_Sun/d_star)^2
        #     to get the minimization to converge
        init_proxy = proxy[:,proxy_to_fit^proxy_mg]
        scale_fac = (SOLAR_DIST_PC/STELLAR_DIST_PC)**2
        print('initial scale factor:',scale_fac)
        init_proxy[1] = init_proxy[1]*scale_fac

        proxy_alpha,proxy_chi2 = scale_spectrum(ref=gratings['G230L'][:,~stis_mg],spectrum=init_proxy,
                                                        init_factor=1,coarse_range=3,fine_range=0.5,end='low')

        proxy_alpha=proxy_alpha*scale_fac
        
        scaled_proxy = np.array([proxy[0],
                                 proxy[1]*proxy_alpha,
                                 proxy[2]*proxy_alpha,
                                 np.ones(len(proxy[0]))*INST_BITS['proxy']])
        
        fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True)
        ax.plot(gratings['G230L'][0],gratings['G230L'][1],label='G230L')
        ax.plot(scaled_proxy[0],scaled_proxy[1],label=r'Scaled proxy: $\alpha$=%0.4e'%(proxy_alpha))
        ax.legend()

        ax.set_xlim([min(proxy[0][proxy_to_fit]),max(gratings['G230L'][0])])

        ax.set_xlabel(r'Wavelength [$\AA$]')
        ax.set_ylabel(r'Flux density [erg s$^{-1}$ cm${-2}$ $\AA^{-1}$]')

        print('EXCEPTION: SOLAR PROXY')
        print('Acutal scale factor:\t%0.4e'%(proxy_alpha))
        
    else:
        proxy_alpha,proxy_chi2 = scale_spectrum(ref=gratings['G230L'][:,~stis_mg],
                                                             spectrum=proxy[:,proxy_to_fit^proxy_mg],
                                                        init_factor=0.01,coarse_range=3,fine_range=0.5,end='low')

        scaled_proxy = np.array([proxy[0],
                                 proxy[1]*proxy_alpha,
                                 proxy[2]*proxy_alpha,
                                 np.ones(len(proxy[0]))*INST_BITS['proxy']])
        
        
        fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True);
        ax.plot(gratings['G230L'][0],gratings['G230L'][1],label='G230L');
        ax.plot(scaled_proxy[0],scaled_proxy[1],label=r'Scaled proxy: $\alpha$=%0.4e'%(proxy_alpha));
        ax.legend();

        ax.set_xlim([min(proxy[0][proxy_to_fit]),max(gratings['G230L'][0])]);

        ax.set_xlabel(r'Wavelength [$\AA$]');
        ax.set_ylabel(r'Flux density [erg s$^{-1}$ cm${-2}$ $\AA^{-1}$]');
        
plt.savefig('./%s/proxy_scaling.png'%(STAR))

Fitting region:	1742.5-2999.5
--- RESULTS ---
Best scale factor:	2.3595e-05
Minimum chi square:	2.0217e+02


# Get EUV bins

In [12]:
#### EUV bin relations from Linsky et al (2014). Use integrated lyman-a flux to calculate 100-1170A flux density

print('Stellar type:',STELLAR_TYPE)
print('Distance: %dpc'%(STELLAR_DIST_PC))

### Get Lya flux & scale to 1AU ###
lya_scale_fac = (STELLAR_DIST_PC*pc2au)**2
int_lya_flux = np.trapz(x=lya[0],y=lya[1])
# Proxy lya flux
#int_lya_flux = np.trapz(x=proxy[0][(proxy[0] > 1214) & (proxy[0] < 1217.4)],y=proxy[1][(proxy[0] > 1214) & (proxy[0] < 1217.4)]*proxy_alpha)
print('Integrated lya flux:\t\t%0.4e erg/s/cm^2'%(int_lya_flux))

int_lya_flux = int_lya_flux*lya_scale_fac
print('Integrated lya flux at 1AU:\t%0.4e erg/s/cm^2'%(int_lya_flux))

# Use Linsky 2014 relations for EUV bins. Relations hold for F-M stars
SUPPORTED_STELLAR_TYPES = ['M','K','G','F']

# Bins 100-400 are different for M stars and F-K stars
if(STELLAR_TYPE.upper() in SUPPORTED_STELLAR_TYPES):
    if(STELLAR_TYPE.upper()=='M'):
        bin_100_200 = 10**(-0.491)*int_lya_flux/100/lya_scale_fac
        bin_200_300 = 10**(-0.548)*int_lya_flux/100/lya_scale_fac
        bin_300_400 = 10**(-0.602)*int_lya_flux/100/lya_scale_fac

    elif(STELLAR_TYPE.upper()=='K' or STELLAR_TYPE.upper()=='F' or STELLAR_TYPE.upper()=='G'):
        bin_100_200 = 10**(-1.357)*int_lya_flux**(0.344+1)/100/lya_scale_fac
        bin_200_300 = 10**(-1.300)*int_lya_flux**(0.309+1)/100/lya_scale_fac
        bin_300_400 = 10**(-0.882)*int_lya_flux/100/lya_scale_fac
    else:
        print('How did you get here? Stellar type must be one of: \'M\' \'K\' \'G\' \'F\'')

    # Bins 400-1170 are the same for all stars.
    bin_400_500 = 10**(-2.294)*int_lya_flux**(0.258+1)/100/lya_scale_fac
    bin_500_600 = 10**(-2.098)*int_lya_flux**(0.572+1)/100/lya_scale_fac
    bin_600_700 = 10**(-1.920)*int_lya_flux**(0.240+1)/100/lya_scale_fac
    bin_700_800 = 10**(-1.894)*int_lya_flux**(0.518+1)/100/lya_scale_fac
    bin_800_912 = 10**(-1.811)*int_lya_flux**(0.764+1)/112/lya_scale_fac
    bin_912_1170 = 10**(-1.004)*int_lya_flux**(0.065+1)/258/lya_scale_fac

    bin_edges = np.array([100,200,300,400,500,600,700,800,912,1170])
    bin_fluxes = np.array([bin_100_200,bin_200_300,bin_300_400,bin_400_500,
                          bin_500_600,bin_600_700,bin_700_800,bin_800_912,bin_912_1170])

    # Define a new wavelength array for the EUV bins to replace the proxy data
    euv_reg = (proxy[0] > 100) & (proxy[0] < 1170)
    EUV_wave_array = proxy[0][euv_reg]
    EUV_flux_array = np.ones(len(EUV_wave_array))
    EUV_inst_array = np.ones(len(EUV_wave_array))*INST_BITS['EUV']

    for i in range(len(bin_fluxes)):
        print('Bin %d-%d:\t%0.4e erg/s/cm^2/A'%(bin_edges[i],bin_edges[i+1],bin_fluxes[i]))
        current_bin = (EUV_wave_array >= bin_edges[i]) & (EUV_wave_array < bin_edges[i+1])
        EUV_flux_array[current_bin] = bin_fluxes[i]

    fig = plt.figure(figsize=(12,8),tight_layout=True);
    ax = fig.add_subplot();
    [x.set_linewidth(1.5) for x in ax.spines.values()];
    ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=12);
    ax.tick_params(which='minor',length=4,width=1.5);
    ax.minorticks_on()
    plt.step(EUV_wave_array,EUV_flux_array);
    plt.title('EUV flux bins',fontsize=15);
    plt.yscale('log');
    plt.grid(which='both',axis='both',alpha=0.4);
    plt.xlabel('Wavelength [A]',fontsize=14);
    plt.ylabel('Flux density [erg / s / cm^2 / A]',fontsize=14);
    
else:
    print('Unsupported stellar type. Skipping EUV bins')
    
    
EUV_data = np.array([EUV_wave_array,
                     EUV_flux_array,
                     np.zeros(len(EUV_wave_array)),
                     EUV_inst_array])

Stellar type: F
Distance: 410pc
Integrated lya flux:		2.8508e-15 erg/s/cm^2
Integrated lya flux at 1AU:	2.0389e+01 erg/s/cm^2
Bin 100-200:	3.5351e-18 erg/s/cm^2/A
Bin 200-300:	3.6272e-18 erg/s/cm^2/A
Bin 300-400:	3.7408e-18 erg/s/cm^2/A
Bin 400-500:	3.1535e-19 erg/s/cm^2/A
Bin 500-600:	1.2763e-18 erg/s/cm^2/A
Bin 600-700:	7.0668e-19 erg/s/cm^2/A
Bin 700-800:	1.7347e-18 erg/s/cm^2/A
Bin 800-912:	3.9366e-18 erg/s/cm^2/A
Bin 912-1170:	1.3319e-18 erg/s/cm^2/A


In [21]:
plt.plot(scaled_proxy[0],scaled_proxy[1],c='C1')

binRegs = np.array([[100,200],[200,300],[300,400],
                    [400,500],[500,600],[600,700],
                    [700,800],[800,912],[912,1170]],ndmin=2)
for reg in binRegs:
    print(reg)
    reg_to_int = (scaled_proxy[0] > reg[0]) & (scaled_proxy[0] < reg[1])
    bin_flux = np.trapz(x=scaled_proxy[0][reg_to_int],y=scaled_proxy[1][reg_to_int])/(reg[1]-reg[0])
    plt.plot([reg[0],reg[1]],[bin_flux,bin_flux],c='C2')
    
legend_lines = [Line2D([0],[0],color='C0'),
               Line2D([0],[0],color='C1'),
               Line2D([0],[0],color='C2')]
legend_handles = ['EUV estimation','Scaled proxy','Integrated proxy bins']
plt.legend(legend_lines,legend_handles,fontsize=14)

[100 200]
[200 300]
[300 400]
[400 500]
[500 600]
[600 700]
[700 800]
[800 912]
[ 912 1170]


# Get X-ray spectrum (somewhat obsolete since we're using PIMMS now)

In [13]:
# get_aeff retrieves effective area from arf file. Only use this if you did not use 'setplot area' in XSPEC
get_aeff = False


try:
    if(get_aeff):
        d = 3.09e18*10 # distance in cm

        with fits.open('./L-98-59/XMM/flare_analysis/F301/src_quiesfilt_0.3-10kev_arf.fits') as hdul:
            resp_energy = hdul[1].data['ENERG_LO']
            a_eff = hdul[1].data['SPECRESP']


        closest_energy = np.zeros(len(x_ray_obs['energy']),dtype=int)


        #### Get the closest energy value to the observed bins & divide by the effective area at that energy
        for i in range(len(x_ray_obs['energy'])):
            closest_energy[i] = int(np.argmin(np.abs(resp_energy-x_ray_obs['energy'][i])))
            print('Energy:\t%0.2fkeV\nA_eff:\t%0.1fcm^2'%(resp_energy[closest_energy[i]],a_eff[closest_energy[i]]))

        fig,ax = plt.subplots(1,1,tight_layout=True,figsize=(9,7));
        [x.set_linewidth(1.2) for x in ax.spines.values()]
        ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=12);
        ax.tick_params(which='minor',length=4,width=1.0);
        ax.minorticks_on();
        ax.plot(APEC_model['energy'],APEC_model['model_flux'],label='APEC plasma model',zorder=-1);
        ax.errorbar(x=x_ray_obs['energy'],y=x_ray_obs['counts'],
                    xerr=x_ray_obs['bin_width'],yerr=x_ray_obs['counts_err'],
                    linestyle='',label='X-ray observation',zorder=0);
        ax.set_ylabel(r'Counts / s / cm$^2$ / keV',fontsize=14);
        ax.set_xlabel('Energy [keV]',fontsize=14);
        ax.legend(fontsize=12)

    #### TODO
    #### COME UP WITH A WAY TO AUTOMATE RETRIEVAL OF X-RAY DATA & XSPEC MODELS

    ####### x-ray model CSV contains ########
    # Energy [keV]  |  energy bin width [keV]  |  total counts  |  counts error  |  photon flux [photon/s/cm^2/keV]
    #########################################


    x_ray_data = pd.read_csv('./L-98-59/XMM/flare_analysis/F301/x_ray_model.csv',delimiter=',')

    end_of_data = 0
    start_of_APEC = 0
    for row,index in zip(x_ray_data['energy'],range(len(x_ray_data['energy']))):
        print(index,row)
        if(row.lower()=='break'):
            # Subtract 1 from index because pandas range is inclusive(?)
            end_of_data = index-1
            start_of_APEC = index+1
            print('Data ends at:',index)
            break

    x_ray_obs = x_ray_data.loc[0:end_of_data].astype(float)
    print(x_ray_obs)
    APEC_model = x_ray_data.loc[start_of_APEC::].astype(float)
    print(APEC_model)


    obs_energy = np.array(x_ray_obs['energy'])[::-1]

    obs_wave = np.array(h*c/obs_energy)
    obs_flux = np.array(x_ray_obs['counts'][::-1]*obs_energy)*erg*obs_energy**2/(h*c)
    obs_err = np.array(x_ray_obs['counts_err'][::-1]*obs_energy*erg*obs_energy**2/(h*c))


    apec_energy = np.array(APEC_model['energy'])[::-1]
    apec_wave = h*c/apec_energy
    apec_flux = np.array(APEC_model['model_flux'][::-1])*erg*apec_energy**2/(h*c)

    fig,ax = plt.subplots(1,1,figsize=(10,7),tight_layout=True)
    ax.errorbar(x=obs_wave,
                y=obs_flux,
                xerr=np.sqrt((-h*c/obs_energy**2)**2*x_ray_obs['bin_width'][::-1]**2),
                yerr=obs_err,linestyle='',label=STAR,zorder=2)
    ax.plot(apec_wave,apec_flux,label='APEC',zorder=1)
    ax.plot(wave,flux*proxy_alpha,label='Scaled proxy',zorder=0)
    ax.set_xlabel('Wavelength [Å]')
    ax.set_ylabel('erg / s / cm^2 / A')

    ax.set_yscale('log')
    ax.minorticks_on()
    ax.legend()

    fig.suptitle(STAR)

    apec_region1 = (apec_wave > 5.) & (apec_wave < obs_wave[1])
    apec_region2 = (apec_wave > obs_wave[-1]) & (apec_wave < 100)
    apec_region = apec_region1+apec_region2

    apec_data = np.array([apec_wave[apec_region],
                          apec_flux[apec_region],
                          np.zeros(np.sum(apec_region)),
                          np.ones(np.sum(apec_region))*INST_BITS['APEC']])


    obs_data = np.array([obs_wave[1::],
                         obs_flux[1::],
                         obs_err[1::],
                         np.ones(len(obs_wave[1::]))*INST_BITS['X-RAY']])

    xray_data = np.concatenate((apec_data,obs_data),axis=1)
except:
    x_ray_reg = (scaled_proxy[0] > 5) & (scaled_proxy[0] < 100)
    xray_data = scaled_proxy[:,x_ray_reg]
    print('Something went wrong so we used the scaled proxy instead!')

Something went wrong so we used the scaled proxy instead!


# Put the pieces together

In [14]:
# If there is a gap between G230L & G430L, fill it with the proxy
if(gratings['G430L'][0][0] > gratings['G230L'][0][-1]):
    #### Obs UV data
    # gratings['G230L'],  gratings['G430L']
    STIS_data = np.concatenate((gratings['G140L'],gratings['G230L'],gratings['G430L']),axis=1)
    #STIS_data = gratings['G430L']
    plt.plot(STIS_data[0],STIS_data[1],label='STIS')

    #### Proxy data
    # scaled_proxy
    proxy_region1 = (scaled_proxy[0] > 1170.) & (scaled_proxy[0] < gratings['G140L'][0][0])
    proxy_region2 = (scaled_proxy[0] > gratings['G140L'][0][-1]) & (scaled_proxy[0] < gratings['G230L'][0][0])
    proxy_region3 = (scaled_proxy[0] > gratings['G230L'][0][-1]) & (scaled_proxy[0] < gratings['G430L'][0][0])
    proxy_region = proxy_region1+proxy_region2+proxy_region3
    proxy_data = scaled_proxy[:,proxy_region]

    plt.plot(proxy_data[0],proxy_data[1])

    #### BT-Settl
    # scaled_settl
    settl_region = (scaled_settl[0] > gratings['G430L'][0][-1]) & (scaled_settl[0] < 55000)

    settl_data = scaled_settl[:,settl_region]

    plt.plot(settl_data[0],settl_data[1])

    #### EUV data
    # EUV_data
    plt.plot(EUV_data[0],EUV_data[1])

    #### X-ray data
    # xray_data
    plt.errorbar(x=xray_data[0],y=xray_data[1],yerr=xray_data[2])

    # Combine each data set into a single array
    combined_spectrum_nolines = np.concatenate((xray_data,
                                               EUV_data,
                                               proxy_data,
                                               STIS_data,
                                               settl_data),axis=1)

    # Get the sorted indices of the wavelength array using argsort and use that to sort the entire combined array
    combined_spectrum_nolines = combined_spectrum_nolines[:,np.argsort(combined_spectrum_nolines[0])]


    # Turn the combined_spectrum_nolines into a Spectrum1D for ease of inserting emission lines
    combined_spec1d = Spectrum1D(spectral_axis=combined_spectrum_nolines[0]*u.AA,
                                 flux=combined_spectrum_nolines[1]*u.Unit('erg cm-2 s-1 AA-1'),
                                 uncertainty=StdDevUncertainty(combined_spectrum_nolines[2]))
# If there is NO gap between G230L and G430L, use all of G430L
else:
    #### Obs UV data
    # gratings['G230L'],  gratings['G430L']
    g230_to_use = gratings['G230L'][0] < gratings['G430L'][0][0]
    STIS_data = np.concatenate((gratings['G230L'][:,g230_to_use],gratings['G430L']),axis=1)
    plt.plot(STIS_data[0],STIS_data[1],label='STIS')

    #### Proxy data
    # scaled_proxy
    proxy_region = (scaled_proxy[0] > 1170.) & (scaled_proxy[0] < gratings['G230L'][0][0])

    proxy_data = scaled_proxy[:,proxy_region]

    plt.plot(proxy_data[0],proxy_data[1])

    #### BT-Settl
    # scaled_settl
    settl_region = (scaled_settl[0] > gratings['G430L'][0][-1]) & (scaled_settl[0] < 55000)

    settl_data = scaled_settl[:,settl_region]

    plt.plot(settl_data[0],settl_data[1])

    #### EUV data
    # EUV_data
    plt.plot(EUV_data[0],EUV_data[1])

    #### X-ray data
    # xray_data
    plt.errorbar(x=xray_data[0],y=xray_data[1],yerr=xray_data[2])

    # Combine each data set into a single array
    combined_spectrum_nolines = np.concatenate((xray_data,
                                               EUV_data,
                                               proxy_data,
                                               STIS_data,
                                               settl_data),axis=1)

    # Get the sorted indices of the wavelength array using argsort and use that to sort the entire combined array
    combined_spectrum_nolines = combined_spectrum_nolines[:,np.argsort(combined_spectrum_nolines[0])]


    # Turn the combined_spectrum_nolines into a Spectrum1D for ease of inserting emission lines
    combined_spec1d = Spectrum1D(spectral_axis=combined_spectrum_nolines[0]*u.AA,
                                 flux=combined_spectrum_nolines[1]*u.Unit('erg cm-2 s-1 AA-1'),
                                 uncertainty=StdDevUncertainty(combined_spectrum_nolines[2]))

result = np.copy(combined_spectrum_nolines)

# Replace Lya

In [15]:
# Replace Lya first
lya_region = (result[0] > lya[0][0]) & (result[0] < lya[0][-1])
result = spf.replace(result,lya)

fig,ax = plt.subplots(3,1,figsize=(7,7),tight_layout=True,sharex=True)
spf.setup_axis(ax)
ax[0].plot(result[0],result[1])
ax[1].plot(result[0],result[1]/result[2])
ax[2].plot(result[0],result[3])

C:\Users\pabe9855\AppData\Local\Temp/ipykernel_16352/2742923712.py:8: RuntimeWarning: divide by zero encountered in true_divide
  ax[1].plot(result[0],result[1]/result[2])


# Create Emission Lines

Have to do this one by hand for each star, unfortunately

1) Update line dictionary with correct fluxes

2) Update inst_bit with correct instrument

In [38]:
# Create a dictionary of lines in the form {line center:integrated flux}
lines = {1238.8:5.14e-18,1242.8:1.099e-17,1393.76:1.32e-17,1402.77:8.13e-18}
instruments = ['G140L','G140L','G140L','G140L']

result = np.copy(combined_spectrum_nolines)
bkg_poly_array = [] # Store the background polynomials to replace regions where the emission doesn't line up well with the continuum

# For each line,flux combo, make a gaussian emission line and replace that portion of the spectrum.
for line,flux,inst in zip(lines.keys(),lines.values(),instruments):
    
    print(line)
    
    inst_bit = INST_BITS[inst]
    
    wave_,flux_,best_amp,int_flux = make_emission_line(line,flux)
    
    # Get a background continuum to add emission line to
    plt.figure()
    plt.plot(scaled_proxy[0],scaled_proxy[1])
    
    line_points = np.asarray(plt.ginput(n=-1,timeout=0))[-2::]
    bkg_points = np.asarray(plt.ginput(n=-1,timeout=0))[-2::]
    
    line_reg = (result[0] > line_points[0][0]) & (result[0] < line_points[1][0])
    bkg_reg = (result[0] > bkg_points[0][0]) & (result[0] < bkg_points[1][0])
    
    wave_reg = bkg_reg^line_reg
    
    if(line < 1300):
        deg = 2
    else:
        deg = 1
        
    bkg_pparams = np.polyfit(x=result[0][wave_reg],y=result[1][wave_reg],deg=deg)
    bkg_poly = np.poly1d(bkg_pparams)
    
    bkg_poly_array.append(bkg_poly)
    
    line_model = np.array([wave_,
                           flux_+bkg_poly(wave_),
                           np.zeros(len(wave_)),
                           np.ones(len(wave_))*inst_bit])
    
    plt.plot(result[0][wave_reg],bkg_poly(result[0][wave_reg]))
    plt.plot(line_model[0],line_model[1])
    
    result = spf.replace(result,line_model)
    
    
plt.figure(num='replaced')
plt.plot(result[0],result[1])

1238.8
1242.8
1393.76
1402.77


In [41]:
# Replace regions where the line doesn't fit in well with the continuum using the background fits
reg1 = [1239.8,1241.64]
reg2 = [1244.11,1245.95]

x = np.arange(reg2[0],reg2[1],step=0.5)

bkg_replace = np.array([x,
                       bkg_poly_array[1](x),
                       np.zeros(len(x)),
                       np.ones(len(x))*INST_BITS['G140L']])

result = spf.replace(result,bkg_replace)

In [42]:
plt.figure()
ax1 = plt.subplot(211)
spf.setup_axis(ax1)
plt.plot(gratings['G140L'][0],gratings['G140L'][1])
plt.plot(scaled_proxy[0],scaled_proxy[1])
plt.plot(result[0],result[1])
ax2 = plt.subplot(212,sharex=ax1)
spf.setup_axis(ax2)
ax2.plot(result[0],result[3])
plt.plot(gratings['G140L'][0],gratings['G140L'][1]/gratings['G140L'][2])
plt.ylim([-1,6])
plt.axhline(y=3,c='k',linestyle='--')

# Replace emission regions with STIS (WASP-77A, L 98-59, HD 149026)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(6,6),sharex=True)
spf.setup_axis(ax)
#ax[0].plot(gratings['G140L'][0],gratings['G140L'][1])
#ax[1].plot(gratings['G140L'][0],gratings['G140L'][1]/gratings['G140L'][2])
#ax[0].plot(gratings['G230L'][0],gratings['G230L'][1])
#ax[1].set_ylim(-1,10)
#ax[1].axhline(y=3,linestyle='--',color='k')

In [ ]:
ax[0].plot(result[0],result[1])
ax[1].plot(result[0],result[1]/result[2])
ax[1].set_ylim(-1,10)

In [ ]:
#G140M_lines = np.array([[]])

#G140L_lines = np.array([[1173.5,1180.1],
#                        [1263.1,1268.1],
#                        [1302.6,1313.0],
#                        [1387.3,1405.6],
#                        [1542.7,1563.9]])

G230L_lines = np.array([[2786.0,2815.0]])

#result = np.copy(combined_spectrum_nolines)
try:
    for line in G140M_lines:
        region = (gratings['G140M'][0] > line[0]) & (gratings['G140M'][0] < line[1])

        result = spf.replace(result,gratings['G140M'][:,region])
except:
    pass

try:
    for line in G140L_lines:
        region = (gratings['G140L'][0] > line[0]) & (gratings['G140L'][0] < line[1])

        result = spf.replace(result,gratings['G140L'][:,region])
except:
    pass

try:
    for line in G230L_lines:
        region = (gratings['G230L'][0] > line[0]) & (gratings['G230L'][0] < line[1])

        result = spf.replace(result,gratings['G230L'][:,region])
except:
    pass


plt.figure()
plt.plot(combined_spectrum_nolines[0],combined_spectrum_nolines[1])
plt.plot(result[0],result[1])

# Make FITS file

In [43]:
###### Double check plot to make sure you're happy!! 

fig,ax = plt.subplots(3,1,figsize=(9,9),tight_layout=True,sharex=True)
spf.setup_axis(ax)

pos_flux = result[2] > 0

ax[0].step(result[0],result[1]*1e13,where='mid')
ax[1].step(result[0],result[1]/result[2],where='mid')
ax[2].step(result[0],result[3],where='mid')

ax[0].set_ylabel(r'Flux density [10$^{-13}$ erg / s / cm$^{2}$ / Å]')
ax[1].set_ylabel('SNR')
ax[2].set_ylabel('Instrument bit')

C:\Users\pabe9855\AppData\Local\Temp/ipykernel_16352/1619723731.py:9: RuntimeWarning: divide by zero encountered in true_divide
  ax[1].step(result[0],result[1]/result[2],where='mid')


Text(0, 0.5, 'Instrument bit')

### Mask parts if necessary

In [ ]:
mask = (result[0] > 3384.77) & (result[0] < 3392.4)
region = (result[0] > 3379.8) & (result[0] < 3403.04)
mask_spline = UnivariateSpline(x=result[0][region^mask],
                               y=result[1][region^mask],k=4)
x = np.linspace(3368,3403)
ax[0].plot(x,mask_spline(x)*1e13)

In [ ]:
result[1,mask] = mask_spline(result[0,mask])
ax[0].plot(result[0,region],result[1,region]*1e13)

####### REPLACE EMISSION AT ~1300 FOR HAT-P-26 WITH RMS OF CONTINUUM ###########

region = (result[0] > 1300.7) & (result[0] < 1308.9)
rms_region = (result[0] > 1312) & (result[0] < 1330)
rms = np.sqrt(np.mean(result[1][rms_region]**2))*np.ones(np.sum(region))
result[1,region] = rms
print(rms)

### Write to the file

In [44]:
FILENAME = './%s/%s_combined_x1d_FINAL_FIXG430LSPIKE.fits'%(STAR,STAR)

hdr = fits.Header()
hdr['COMMENT'] = '%s 1D spectrum'%(STAR)

primary = fits.PrimaryHDU(header=hdr)

hduA = fits.BinTableHDU.from_columns([
    fits.Column(name = 'WAVELENGTH',format='D',array=result[0]),
    fits.Column(name = 'FLUX',format='D',array=result[1]),
    fits.Column(name = 'ERROR', format='D',array=result[2]),
    fits.Column(name = 'INST', format='I',array=result[3])])

hdulA = fits.HDUList([primary,hduA])
hdulA.writeto(FILENAME,overwrite=True)

hdulA.close()

# Read & plot FITS file to double check :)

In [ ]:
with fits.open('./%s/%s_combined_x1d_FINAL.fits'%(STAR,STAR)) as hdul:
    data = hdul[1].data
    
    hdul.close()

fig,ax = plt.subplots(3,1,figsize=(7,7),tight_layout=True,sharex=True)
spf.setup_axis(ax)
ax[0].plot(data['WAVELENGTH'],data['FLUX'])
ax[1].plot(data['WAVELENGTH'],data['FLUX']/data['ERROR'])
ax[2].plot(data['WAVELENGTH'],data['INST'])

# Random debugging, testing, etc

## Test Instrument binning

In [ ]:
test_wave = gratings['G140L'][0]
test_flux = gratings['G140L'][1]
test_inst = gratings['G140L'][3]

test_inst[100:102] = INST_BITS['proxy']

ax1 = plt.subplot(211)
plt.plot(test_wave,test_flux)
plt.subplot(212,sharex=ax1)
plt.plot(test_wave,test_inst)

In [ ]:
num_bins = combined_spectrum_nolines[0][-1] - combined_spectrum_nolines[0][0]

test_flux_binned,flux_bins,_ = binned_statistic(x=combined_spectrum_nolines[0],
                                                values=combined_spectrum_nolines[1],
                                                bins=num_bins,statistic='mean')
test_inst_binned,inst_bins,_ = binned_statistic(x=combined_spectrum_nolines[0],
                                                values=combined_spectrum_nolines[3],
                                                bins=num_bins,statistic=get_inst_bins)

In [ ]:
ax1 = plt.subplot(211)
plt.plot(flux_bins[0:-1],test_flux_binned)
plt.subplot(212,sharex=ax1)
plt.plot(inst_bins[0:-1],test_inst_binned)

## GJ 832 Spectrum

In [ ]:
with fits.open('./Proxy stars/GJ 832/hlsp_muscles_multi_multi_gj832_broadband_v22_var-res-sed.fits') as hdul:
    plt.figure()
    plt.semilogy(hdul[1].data['WAVELENGTH'],hdul[1].data['FLUX'])
    
    gjwave = hdul[1].data['WAVELENGTH']
    gjflux = hdul[1].data['FLUX']
    
gj_832_apec = pd.read_csv('./GJ_832_apec.csv')
#
#plt.plot(h*c/gj_832_apec['energy'],gj_832_apec['model_flux']*erg*gj_832_apec['energy']**2/(h*c)*(9.35e-15/9.365e-10))
#
plt.figure()
plt.plot(np.diff(gjwave[(gjwave > 20) & (gjwave < 100)]))

In [ ]:
bins = np.arange(combined_spectrum_nolines[0][0],combined_spectrum_nolines[0][-1],step=1)
bin_ind = np.digitize(combined_spectrum_nolines[0],bins)

In [ ]:
flux_bins = np.mean(combined_spectrum_nolines[1][[bin_ind==i] for i in bins],axis=0)

In [ ]:
g140m_spec1d = Spectrum1D(spectral_axis=gratings['G140M'][0]*u.AA,
                          flux=gratings['G140M'][1]*u.Unit('erg cm-2 s-1 AA-1'),
                          uncertainty=StdDevUncertainty(gratings['G140M'][2]))

In [ ]:
plt.plot(g140m_spec1d.spectral_axis,g140m_spec1d.flux)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(7,7),tight_layout=True)
ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=14)
ax.tick_params(which='minor',length=4,width=1)
ax.minorticks_on()
ax.grid(which='both',alpha=0.5)

ax.errorbar(x=x_ray_obs['energy'],
           y=x_ray_obs['counts'],
           xerr=x_ray_obs['bin_width'],
           yerr=x_ray_obs['counts_err'],linestyle='',marker='.',markersize=3,label='L 98-59 XMM-Newton',zorder=2)
ax.plot(APEC_model['energy'],APEC_model['model_flux'],zorder=1,lw=0.8,label='APEC')
ax.set_xlabel('Energy [keV]',fontsize=14)
ax.set_ylabel(r'counts s$^{-1}$ cm$^{-2}$ keV$^{-1}$',fontsize=14)
ax.legend(fontsize=14)

In [ ]:
FILENAME = './L-98-59/XMM/flare_analysis/F301/L-98-59_xray.fits'

hdr = fits.Header()
hdr['COMMENT'] = 'L 98-59 XMM-Newton spectrum'

primary = fits.PrimaryHDU(header=hdr)

hduA = fits.BinTableHDU.from_columns([
    fits.Column(name = 'ENERGY',format='D',array=x_ray_obs['energy']),
    fits.Column(name = 'BIN_WIDTH',format='D',array=x_ray_obs['bin_width']),
    fits.Column(name = 'PHOTFLUX',format='D',array=x_ray_obs['counts']),
    fits.Column(name = 'ERROR', format='D',array=x_ray_obs['counts_err'])])

hdulA = fits.HDUList([primary,hduA])
hdulA.writeto(FILENAME,overwrite=True)

hdulA.close()

In [ ]:
FILENAME = './L-98-59/XMM/flare_analysis/F301/L-98-59_APEC.fits'

hdr = fits.Header()
hdr['COMMENT'] = 'L 98-59 APEC Model'

primary = fits.PrimaryHDU(header=hdr)

hduA = fits.BinTableHDU.from_columns([
    fits.Column(name = 'ENERGY',format='D',array=APEC_model['energy']),
    fits.Column(name = 'PHOTFLUX',format='D',array=APEC_model['model_flux'])])
hdulA = fits.HDUList([primary,hduA])
hdulA.writeto(FILENAME,overwrite=True)

hdulA.close()

In [ ]:
with fits.open('./L-98-59/XMM/flare_analysis/F301/L-98-59_xray.fits') as hdul:
    xray = hdul[1].data
    
    hdul.close()
    
with fits.open('./L-98-59/XMM/flare_analysis/F301/L-98-59_APEC.fits') as hdul:
    apec = hdul[1].data
    
    hdul.close()

In [ ]:
plt.style.use('dark_background')

fig,ax = plt.subplots(1,1,figsize=(7,7),tight_layout=True)
[x.set_linewidth(1.5) for x in ax.spines.values()]
ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=14)
ax.tick_params(which='minor',length=4,width=1)
ax.minorticks_on()
ax.grid(which='both',alpha=0.2)

ax.errorbar(x=xray['ENERGY'],
           y=xray['PHOTFLUX']*1000,
           xerr=xray['BIN_WIDTH'],
           yerr=xray['ERROR']*1000,linestyle='',marker='.',markersize=4,elinewidth=2.,label='L 98-59 XMM-Newton',zorder=2,color='r')
ax.plot(apec['ENERGY'],apec['PHOTFLUX']*1000,zorder=1,lw=0.8,label='APEC',c='w')
ax.set_xlabel('Energy [keV]',fontsize=14)
ax.set_ylabel(r'10$^{-3}$ counts s$^{-1}$ cm$^{-2}$ keV$^{-1}$',fontsize=14)
ax.legend(fontsize=14)

In [ ]:
lya_data

In [ ]:
for i in range(len(lya_data)):
    print(lya_data['lya_intrinsic_low_1sig'][i],lya_data['lya_intrinsic_median'][i],lya_data['lya_intrinsic_high_1sig'][i])
    print('Lower 1sig: ',lya_data['lya_intrinsic_median'][i]-lya_data['lya_intrinsic_low_1sig'][i])
    print('Upper 1sig: ',lya_data['lya_intrinsic_high_1sig'][i]-lya_data['lya_intrinsic_median'][i])

In [ ]:
###############################
#  Lya recon figure for paper #
###############################

W77A = 'C:/Users/pabe9855/Desktop/Directories/MEATS/data/WASP-77A/G140M/x1d_coadd.fits'

with fits.open(W77A) as hdul:
    W77A_wave = hdul[1].data['WAVELENGTH']
    W77A_flux = hdul[1].data['FLUX']
    
W77Lya_dir = 'C:/Users/pabe9855/Desktop/Directories/MEATS/data/WASP-77A/lya_reconstruction/WASP-77A_lya_intrinsic_profile.csv'
W77Lya = pd.read_csv(W77Lya_dir)

W77Lya = np.array([W77Lya['wave'],W77Lya['flux']])



fig,ax = plt.subplots(1,2,figsize=(12,6),tight_layout=True)
for axis in ax:
    [x.set_linewidth(1.5) for x in axis.spines.values()]
    axis.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=13)
    axis.tick_params(which='minor',length=4,width=1.0)
    axis.minorticks_on()
    
ax[0].set_ylabel(r'Flux Density [10$^{-14}$ erg  s$^{-1}$ cm$^{-2}$ Å$^{-1}$]',fontsize=15)
ax[0].set_xlabel(r'Wavelength [Å]',fontsize=15)
ax[0].step(gratings['G140M'][0],gratings['G140M'][1]*1e14,label='STIS G140M')
ax[0].step(lya[0],lya[1]*1e14,label=r'L$\alpha$ MCMC reconstruction')
ax[0].legend(fontsize=12,loc='lower right')
ax[0].set_title('HD 149026',fontsize=16)
ax[0].set_xlim([1213.277,1218.190])
ax[0].set_ylim([-0.910,1.940])

ax[1].set_ylabel(r'Flux Density [10$^{-15}$ erg  s$^{-1}$ cm$^{-2}$ Å$^{-1}$]',fontsize=15)
ax[1].set_xlabel(r'Wavelength [Å]',fontsize=15)
ax[1].step(W77A_wave,W77A_flux*1e15,label='STIS G140M')
ax[1].step(W77Lya[0],W77Lya[1]*1e15,label=r'L$\alpha$ MgII estimation')
ax[1].legend(fontsize=12,loc='lower right')
ax[1].set_title('WASP-77A',fontsize=16)
ax[1].set_xlim([1210.478,1220.614])
ax[1].set_ylim([-3.279,6.316])

In [ ]:
with fits.open('./HD-149026/XMM/0763460301/data_prod/src_timefilt_lc.fits') as hdul:
    data = hdul[1].data
    hdul.close()
    
plt.plot(data['TIME'],data['RATE'])

# Extinction

In [ ]:
import dust_extinction
from dust_extinction.averages import GCC09_MWAvg,I05_MWAvg

Rv = 3.1
Ev = 0.04
Av = Rv*Ev
print('Av=',Av)

x=np.arange(913,7000,0.01)*u.AA

ext_model = GCC09_MWAvg()
ext_corr = ext_model.extinguish(x=x,Ebv=Ev)

with fits.open('./WASP-77A/combined_spectrum/WASP-77A_x1d_native_res.fits') as hdul:
    data = hdul[1].data
    hdul.close()
    

# interpolate ext_curve onto wavelength grid
reg = (data['WAVELENGTH'] > x[0]*1/u.AA) & (data['WAVELENGTH'] < x[-1]*1/u.AA)
ext_corr_interp = np.interp(x=data['WAVELENGTH'][reg]*u.AA,xp=x,fp=ext_corr)

fig = plt.figure(figsize=(11,6))
gs = fig.add_gridspec(2,1,height_ratios=(1,3),wspace=0.05,hspace=0.05)
ax_top = fig.add_subplot(gs[0])
ax_bot = fig.add_subplot(gs[1],sharex=ax_top)
spf.setup_axis(ax_top)
spf.setup_axis(ax_bot)
ax_top.plot(data['WAVELENGTH'][reg],1/ext_corr_interp,label='GCC09')
ax_top.plot()
ax_top.set_ylabel(r'Correction factor',fontsize=14)
ax_top.tick_params(labelbottom=False)

ax_bot.semilogy(data['WAVELENGTH'][reg],data['FLUX'][reg]*1e14,label='reddened')
ax_bot.semilogy(data['WAVELENGTH'][reg],data['FLUX'][reg]/ext_corr_interp*1e14,label='unreddened')
ax_bot.set_xlabel('Wavelength [Å]',fontsize=14)
ax_bot.set_ylabel('Flux dens [0$^{-14}$ erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$]',fontsize=14)
ax_bot.legend(fontsize=14)

fig.suptitle('WASP-77A Unreddened (d=105pc, R_v = 3.1, E_v = 0.04, A_v=%0.3f)'%(Av),fontsize=14)

In [ ]:
import dust_extinction
from dust_extinction.averages import GCC09_MWAvg,I05_MWAvg

Rv = 3.1
Ev = -0.43
Av = Rv*Ev
print('Av=',Av)

x=np.arange(913,7000,0.01)*u.AA

ext_model = GCC09_MWAvg()
ext_corr = ext_model.extinguish(x=x,Av=0.1)

with fits.open('./WASP-127/combined_spectrum/WASP-127_x1d_native_res.fits') as hdul:
    data = hdul[1].data
    hdul.close()
    

# interpolate ext_curve onto wavelength grid
reg = (data['WAVELENGTH'] > x[0]*1/u.AA) & (data['WAVELENGTH'] < x[-1]*1/u.AA)
ext_corr_interp = np.interp(x=data['WAVELENGTH'][reg]*u.AA,xp=x,fp=ext_corr)

fig = plt.figure(figsize=(11,6))
gs = fig.add_gridspec(2,1,height_ratios=(1,3),wspace=0.05,hspace=0.05)
ax_top = fig.add_subplot(gs[0])
ax_bot = fig.add_subplot(gs[1],sharex=ax_top)
spf.setup_axis(ax_top)
spf.setup_axis(ax_bot)
ax_top.plot(data['WAVELENGTH'][reg],1/ext_corr_interp,label='GCC09')
ax_top.plot()
ax_top.set_ylabel(r'Correction factor',fontsize=14)
ax_top.tick_params(labelbottom=False)

ax_bot.semilogy(data['WAVELENGTH'][reg],data['FLUX'][reg]*1e14,label='reddened')
ax_bot.semilogy(data['WAVELENGTH'][reg],data['FLUX'][reg]/ext_corr_interp*1e14,label='unreddened')
ax_bot.set_xlabel('Wavelength [Å]',fontsize=14)
ax_bot.set_ylabel('Flux dens [0$^{-14}$ erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$]',fontsize=14)
ax_bot.legend(fontsize=14)

fig.suptitle('WASP-77A Unreddened (d=105pc, R_v = 3.1, E_v = %0.3f, A_v=%0.3f)'%(Ev,Av),fontsize=14)

# Test Ly$\alpha$ flux along wing drop

In [ ]:
fig, ax = plt.subplots(1,1,tight_layout=True)
spf.setup_axis(ax)
ax.semilogy(data['WAVELENGTH'],data['FLUX'])

In [ ]:
print('Click points for spline 1')
points_for_spline1 = np.array(fig.ginput(n=-1,timeout=-1))
print('Click points for spline 2')
points_for_spline2 = np.array(fig.ginput(n=-1,timeout=-1))

spline_lam1 = points_for_spline1[:,0]
spline_flux1 = points_for_spline1[:,1]
spline_lam2 = points_for_spline2[:,0]
spline_flux2 = points_for_spline2[:,1]

In [ ]:
lam_array1 = np.linspace(spline_lam1[0],spline_lam1[-1],1000)
lam_array2 = np.linspace(spline_lam2[0],spline_lam2[-1],1000)

spline1 = CubicSpline(spline_lam1,spline_flux1)
spline2 = CubicSpline(spline_lam2,spline_flux2)

ax.plot(lam_array1,spline1(lam_array1))
ax.plot(lam_array2,spline2(lam_array2))

In [ ]:
wave_reg1 = (data['WAVELENGTH'] >= lam_array1[0]) & (data['WAVELENGTH'] <= lam_array1[-1])
wave_reg2 = (data['WAVELENGTH'] >= lam_array2[0]) & (data['WAVELENGTH'] <= lam_array2[-1])
missing_flux = (np.trapz(x=lam_array1,y=spline1(lam_array1))+np.trapz(x=lam_array2,y=spline2(lam_array2))-
                np.trapz(x=data['WAVELENGTH'][wave_reg1],y=data['FLUX'][wave_reg1])+
                np.trapz(x=data['WAVELENGTH'][wave_reg2],y=data['FLUX'][wave_reg2]))

tot_reg = (data['WAVELENGTH'] >= lam_array1[0]) & (data['WAVELENGTH'] <= lam_array2[-1])

print(missing_flux)
tot_flux = np.trapz(x=data['WAVELENGTH'][tot_reg],y=data['FLUX'][tot_reg])
print(tot_flux)
print('------Flux ratio------')
print(missing_flux/tot_flux)

# Get solar FWHM & convolve

In [ ]:
solar_dat = readsav('./Proxy stars/Solar/solar-data.idlsav')
solar_wave = solar_dat['wave']*10
solar_flux = solar_dat['flux']*100
plt.figure()
plt.plot(solar_wave,solar_flux)

In [ ]:
def gaussian(x,a,mu,sig,o):
    return a*np.exp(-1/2*((x-mu)/sig)**2)+o

from scipy.optimize import curve_fit

In [ ]:
print('Click continuum, peak, half-max')
line_points = plt.ginput(n=-1,timeout=0)

In [ ]:
cont1 = 606.
cont2 = 614.
line_points = np.asarray(line_points)
reg = (solar_wave > cont1) & (solar_wave < cont2)

a = line_points[1,1]
o = line_points[0,1]
mu = line_points[1,0]
sig = np.abs(line_points[2,0]-line_points[1,0])/2.355

gauss_fit,cov = curve_fit(f=gaussian,xdata=solar_wave[reg],ydata=solar_flux[reg],p0=[a,mu,sig,o])
print(gauss_fit)
print('FWHM = %0.4f'%(2.355*gauss_fit[2]))

In [ ]:
x_new = np.linspace(cont1,cont2,1000)
plt.plot(x_new,gaussian(x_new,gauss_fit[0],gauss_fit[1],gauss_fit[2],gauss_fit[3]))

#### Solar FWHM based on a few points

In [ ]:
avg_fwhm=np.average((1.8,1.13,1.59,1.3,1.05,1.21,1.50))
print(avg_fwhm)

In [ ]:
convolved_l98 = convolve_fft(data['FLUX'],Gaussian1DKernel(stddev=1.3685714/2.355))

In [ ]:
fig,ax = plt.subplots(1,1,tight_layout=True)
spf.setup_axis(ax,xlabel='Wavelength [Å]',ylabel=r'Flux density [10$^{-14}$ erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$]')
ax.plot(data['WAVELENGTH'],convolved_l98*1e14)

# FUV / NUV

In [ ]:
fuv_reg = (data['WAVELENGTH'] > 1350) & (data['WAVELENGTH'] < 1780)

nuv_reg = (data['WAVELENGTH'] > 1770) & (data['WAVELENGTH'] < 2730)

fuv_wave = data['WAVELENGTH'][fuv_reg]
fuv_flux = data['FLUX'][fuv_reg]
fuv_err = data['ERROR'][fuv_reg]
nuv_wave = data['WAVELENGTH'][nuv_reg]
nuv_flux = data['FLUX'][nuv_reg]
nuv_err = data['ERROR'][nuv_reg]

trials = 10000
fuv_flux_trials = np.zeros(trials)
nuv_flux_trials = np.zeros(trials)

for i in range(trials):
    fuv_temp_flux_array = np.random.normal(loc=fuv_flux,scale=fuv_err)
    fuv_flux_trials[i] = np.trapz(x=fuv_wave,y=fuv_temp_flux_array)
    
    nuv_temp_flux_array = np.random.normal(loc=nuv_flux,scale=nuv_err)
    nuv_flux_trials[i] = np.trapz(x=nuv_wave,y=nuv_temp_flux_array)
    
fuv = np.mean(fuv_flux_trials)
fuv_err = np.std(fuv_flux_trials)

nuv = np.mean(nuv_flux_trials)
nuv_err = np.std(nuv_flux_trials)

print('%0.4e +/- %0.4e'%(fuv/nuv,fuv/nuv*np.sqrt((fuv_err/fuv)**2+(nuv_err/nuv)**2)))